# ASSIGNMENT:

# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
import sys
sys.executable

'/usr/bin/python3'

In [3]:
%env TARGET=10.11.1.74

env: TARGET=10.11.1.74


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep ppp

if not len(result)>0:
    print("Not connected!")
#     input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

Unable to find image 'my-other-github-account/rustscan:latest' locally
docker: Error response from daemon: pull access denied for my-other-github-account/rustscan, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.
See 'docker run --help'.


# Initial Nmap:

In [7]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.11.1.74


In [8]:
!ping -c 5 $TARGET

PING 10.11.1.74 (10.11.1.74) 56(84) bytes of data.
64 bytes from 10.11.1.74: icmp_seq=1 ttl=63 time=1142 ms
64 bytes from 10.11.1.74: icmp_seq=2 ttl=63 time=203 ms
64 bytes from 10.11.1.74: icmp_seq=3 ttl=63 time=312 ms

--- 10.11.1.74 ping statistics ---
5 packets transmitted, 3 received, 40% packet loss, time 4012ms
rtt min/avg/max/mdev = 202.840/552.251/1141.610/419.128 ms, pipe 2


# Quick Scan:

In [9]:
!echo sudo nmap -vv -oA scans/quick/quick_scan $TARGET
!echo
!sudo nmap -vv -oA scans/quick/quick_scan $TARGET

sudo nmap -vv -oA scans/quick/quick_scan 10.11.1.74

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-20 18:33 PDT
Initiating Ping Scan at 18:33
Scanning 10.11.1.74 [4 ports]
Completed Ping Scan at 18:33, 3.06s elapsed (1 total hosts)
Nmap scan report for 10.11.1.74 [host down, received no-response]
Read data files from: /usr/bin/../share/nmap
Note: Host seems down. If it is really up, but blocking our ping probes, try -Pn
Nmap done: 1 IP address (0 hosts up) scanned in 3.14 seconds
           Raw packets sent: 8 (304B) | Rcvd: 1379 (55.319KB)


# All Ports:

In [ ]:
!echo sudo nmap -sC -p- $TARGET
!sudo nmap -sC -p- $TARGET

In [ ]:
!echo sudo nmap -p- -sV -O -vv -oA  scans/tcp/tcp_scan $TARGET
!echo
!sudo nmap -p- -sV -O -vv -oA  scans/tcp/tcp_scan $TARGET

sudo nmap -p- -sV -O -vv -oA scans/tcp/tcp_scan 10.11.1.74

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-20 18:33 PDT
NSE: Loaded 45 scripts for scanning.
Initiating Ping Scan at 18:33
Scanning 10.11.1.74 [4 ports]
Completed Ping Scan at 18:33, 1.49s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 18:33
Completed Parallel DNS resolution of 1 host. at 18:33, 0.01s elapsed
Initiating SYN Stealth Scan at 18:33
Scanning 10.11.1.74 [65535 ports]
Discovered open port 139/tcp on 10.11.1.74
Discovered open port 445/tcp on 10.11.1.74
Discovered open port 22/tcp on 10.11.1.74
Discovered open port 80/tcp on 10.11.1.74
Increasing send delay for 10.11.1.74 from 0 to 5 due to max_successful_tryno increase to 4
SYN Stealth Scan Timing: About 10.44% done; ETC: 18:38 (0:04:26 remaining)
SYN Stealth Scan Timing: About 12.25% done; ETC: 18:41 (0:07:17 remaining)
SYN Stealth Scan Timing: About 14.22% done; ETC: 18:43 (0:09:09 remaining)
SYN Stealth Scan Timing: About 16.46% 

In [ ]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [ ]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

# No Ping:

In [ ]:
!echo sudo nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET
!echo
!sudo nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

In [ ]:
if not PING:
    !sudo nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [ ]:
if PING:
    !sudo nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

In [ ]:
if PING:
    !sudo nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !sudo nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
if PING:
    !sudo nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !sudo nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
from getpass import getpass
import os

if PING:
    !sudo nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !sudo nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
UDP Scan, this takes forever...

from getpass import getpass

!sudo nmap -sU --open -oA scans/udp/udp_scan $TARGET
!sudo nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET